<a href="https://colab.research.google.com/github/brepowell/ML-Contest-Series/blob/main/MLSeriesSupervisedLearningTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Breanna Powell

CSS 539

# To run:
Use an IDE that will allow you to open a Jupyter Notebook.

For example, use Anaconda Navigator to open Visual Studio Code.

You may need to select a kernel to run the program.

I used base(Python 3.10.9)

## Import Libraries

In [1]:
import pandas as pd
import matplotlib as plt

# These libraries are supposed to accelerate some operations
import numexpr
import bottleneck
pd.set_option("compute.use_bottleneck", False)
pd.set_option("compute.use_numexpr", False)

## Data Information

From:
https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households

More Info:
https://innovation.ukpowernetworks.co.uk/projects/low-carbon-london

"Created 9 years ago, updated 2 years ago
Energy consumption readings for a sample of 5,567 London Households that took part in the UK Power Networks led Low Carbon London project between November 2011 and February 2014.

Readings were taken at half hourly intervals. The customers in the trial were recruited as a balanced sample representative of the Greater London population.

The dataset contains energy consumption, in kWh (per half hour), unique household identifier, date and time. The CSV file is around 10GB when unzipped and contains around 167million rows.

Within the data set are two groups of customers. The first is a sub-group, of approximately 1100 customers, who were subjected to Dynamic Time of Use (dToU) energy prices throughout the 2013 calendar year period. The tariff prices were given a day ahead via the Smart Meter IHD (In Home Display) or text message to mobile phone. Customers were issued High (67.20p/kWh), Low (3.99p/kWh) or normal (11.76p/kWh) price signals and the times of day these applied. The dates/times and the price signal schedule is availaible as part of this dataset. All non-Time of Use customers were on a flat rate tariff of 14.228pence/kWh.

The signals given were designed to be representative of the types of signal that may be used in the future to manage both high renewable generation (supply following) operation and also test the potential to use high price signals to reduce stress on local distribution grids during periods of stress.

The remaining sample of approximately 4500 customers energy consumption readings were not subject to the dToU tariff."


## Create a huge dataframe with all hourly info

In [2]:
##############################
# This takes about 2 minutes #
##############################

import glob

def readAllHourlyData():
    # Read all CSV files and concatenate them into a single DataFrame
    path = "data/archive/halfhourly_dataset/halfhourly_dataset/" 

    allFiles = glob.glob(path + "block_*.csv")
    dfList = []
    for filename in allFiles:
        df = pd.read_csv(filename)
        dfList.append(df)
    fullDf = pd.concat(dfList)

    return fullDf

londonData = readAllHourlyData()
londonData.shape

C:\Users\brely\AppData\Local\Temp\ipykernel_16108\1060363303.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_16108\1060363303.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_16108\1060363303.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_16108\1060363303.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_16108\1060363303.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\

(167817021, 3)

In [3]:
londonData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167817021 entries, 0 to 1605346
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   LCLid           object
 1   tstp            object
 2   energy(kWh/hh)  object
dtypes: object(3)
memory usage: 5.0+ GB


In [4]:
londonData.head()
path = "Data/pickles/"
londonData.to_pickle(path + "londonDataInitial.pkl")

# SELECT FEATURES

What features do I want?

* NaN's or Null values - 0 for value or 1 for NaN - NaN could be a blackout
* Day, time, etc.
* Weekends - 0 for weekday or 1 for weekend
* Holidays - 0 for regular day 1 for holiday
* Customer information - affluent, etc.
* Customers served - How many customers were represented on that day? - Might be needed for normalization
* Dip in customers - Could be a sign of an outage - 0 for no dip, 1 for dip
* Blackout signature - mark hours before 0 max energy as signature days?
* Weather Features - add later to see if improves

# ==== SAVE AND LOAD ====
I decided to save the data as a pkl here so that I could just run the cells below this point.

In [5]:
# These libraries are supposed to accelerate some operations
import numexpr
import bottleneck
pd.set_option("compute.use_bottleneck", False)
pd.set_option("compute.use_numexpr", False)

############################################
# This takes about 35 seconds to a minute  #
############################################

# Load

import pandas as pd
path = "Data/pickles/"
londonData = pd.read_pickle(path + "londonDataInitial.pkl")

## Feature: Null / NaN

In [6]:
import numpy as np

# In this dataset, NaN is marked as Null
londonData.replace({'energy(kWh/hh)': {'NULL': np.nan}}, inplace=True)
londonData.replace({'energy(kWh/hh)': {'Null': np.nan}}, inplace=True)
londonData.replace({'energy(kWh/hh)': {'null': np.nan}}, inplace=True)
    
londonData["isNA"] = londonData["energy(kWh/hh)"].isna().astype(int)
print("Number of NaN values: \n{}".format(londonData["isNA"].value_counts(dropna=False)))
londonData.head()


Number of NaN values: 
0    167811461
1         5560
Name: isNA, dtype: int64


,LCLid,tstp,energy(kWh/hh),isNA
0,MAC000002,2012-10-12 00:30:00.0000000,0,0
1,MAC000002,2012-10-12 01:00:00.0000000,0,0
2,MAC000002,2012-10-12 01:30:00.0000000,0,0
3,MAC000002,2012-10-12 02:00:00.0000000,0,0
4,MAC000002,2012-10-12 02:30:00.0000000,0,0


## Feature: Day, Month, Year, Hour, Minute

In [7]:
##############################
# This takes about 2 minutes #
##############################

def extractDayInformation(data):

    # Convert 'tstp' column to datetime
    data['tstp'] = pd.to_datetime(data['tstp'])
    
    # Extract day information
    data['year'] = data['tstp'].dt.year              # 2011, 2012, 2013, 2014
    data['month'] = data['tstp'].dt.month            # 1 to 12
    data['day'] = data['tstp'].dt.day                # 1 to 31
    data['hour'] = data['tstp'].dt.hour              # 1 to 23
    data['minute'] = data['tstp'].dt.minute          # 0 to 54 - I thought this dataset only had half hour increments?
    # data['second'] = data['tstp'].dt.second          # 0 to 59 - Might not need seconds
    data['dayNumber'] = data['tstp'].dt.dayofweek    # 0 to 6 where 0 is Monday and 6 is Sunday
    data['dayString'] = data['tstp'].dt.strftime("%Y-%m-%d") # Will be used for merging other data

    # Get the unix timestamp as well - 1322038800 to 1393545600 - too much information
    # data['unixTimeStamp'] = (data['tstp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

    # Get rid of the tstp column
    data = data.drop(['tstp'], axis=1)
    
    return data

londonData = extractDayInformation(londonData)
londonData.head()


,LCLid,energy(kWh/hh),isNA,year,month,day,hour,minute,dayNumber,dayString
0,MAC000002,0,0,2012,10,12,0,30,4,2012-10-12
1,MAC000002,0,0,2012,10,12,1,0,4,2012-10-12
2,MAC000002,0,0,2012,10,12,1,30,4,2012-10-12
3,MAC000002,0,0,2012,10,12,2,0,4,2012-10-12
4,MAC000002,0,0,2012,10,12,2,30,4,2012-10-12


In [8]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.dayofweek.html
# 0 is Monday; 6 is Sunday; 5 is Saturday
londonData["weekend"] = (londonData["dayNumber"] > 4).astype(int)

In [9]:
# Save

londonData.head()
path = "Data/pickles/"
londonData.to_pickle(path + "londonDataNullandDayInfo.pkl")

# ==== SAVE AND LOAD ====
I decided to save the data as a pkl here so that I could just run the cells below this point.

## Feature: Customers Count Per Day
I will call this CustomerCount

In [1]:
import pandas as pd

# These libraries are supposed to accelerate some operations
import numexpr
import bottleneck
pd.set_option("compute.use_bottleneck", False)
pd.set_option("compute.use_numexpr", False)

In [2]:
############################################
# This takes about 35 seconds to a minute  #
############################################

# Load
path = "Data/pickles/"
londonData = pd.read_pickle(path + "londonDataNullandDayInfo.pkl")

In [3]:
londonData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167817021 entries, 0 to 1605346
Data columns (total 11 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   LCLid           object
 1   energy(kWh/hh)  object
 2   isNA            int32 
 3   year            int64 
 4   month           int64 
 5   day             int64 
 6   hour            int64 
 7   minute          int64 
 8   dayNumber       int64 
 9   dayString       object
 10  weekend         int32 
dtypes: int32(2), int64(6), object(3)
memory usage: 13.8+ GB


In [4]:
allDays = pd.read_pickle(path + "allDays.pkl")

In [5]:
##############################
# This takes about 5 minutes #
##############################

import pandas as pd

# Merge the two dataframes on the "day" and "LCLid" columns
mergedData = londonData.merge(allDays, left_on='dayString', right_on='date', how='inner')

In [6]:
mergedData.head()

,LCLid,energy(kWh/hh),isNA,year,month,day,hour,minute,dayNumber,dayString,weekend,CustomerCount
0,MAC000002,0,0,2012,10,12,0,30,4,2012-10-12,0,5261
1,MAC000002,0,0,2012,10,12,1,0,4,2012-10-12,0,5261
2,MAC000002,0,0,2012,10,12,1,30,4,2012-10-12,0,5261
3,MAC000002,0,0,2012,10,12,2,0,4,2012-10-12,0,5261
4,MAC000002,0,0,2012,10,12,2,30,4,2012-10-12,0,5261


In [7]:
mergedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167788469 entries, 0 to 167788468
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   LCLid           object
 1   energy(kWh/hh)  object
 2   isNA            int32 
 3   year            int64 
 4   month           int64 
 5   day             int64 
 6   hour            int64 
 7   minute          int64 
 8   dayNumber       int64 
 9   dayString       object
 10  weekend         int32 
 11  CustomerCount   int64 
dtypes: int32(2), int64(7), object(3)
memory usage: 15.0+ GB


In [8]:
# Save
path = "Data/pickles/"
mergedData.to_pickle(path + "londonDataCustomerCount.pkl")

### Dip in customers? - Could be a sign of an outage - 0 for no dip, 1 for dip

In [10]:
import pandas as pd

# These libraries are supposed to accelerate some operations
import numexpr
import bottleneck
pd.set_option("compute.use_bottleneck", False)
pd.set_option("compute.use_numexpr", False)

############################################
# This takes about 35 seconds to a minute  #
############################################

# Load
path = "Data/pickles/"
londonData = pd.read_pickle(path + "londonDataCustomerCount.pkl")
londonData.head()

,LCLid,energy(kWh/hh),isNA,year,month,day,hour,minute,dayNumber,dayString,weekend,CustomerCount
0,MAC000002,0,0,2012,10,12,0,30,4,2012-10-12,0,5261
1,MAC000002,0,0,2012,10,12,1,0,4,2012-10-12,0,5261
2,MAC000002,0,0,2012,10,12,1,30,4,2012-10-12,0,5261
3,MAC000002,0,0,2012,10,12,2,0,4,2012-10-12,0,5261
4,MAC000002,0,0,2012,10,12,2,30,4,2012-10-12,0,5261


In [11]:
dipDays = pd.read_csv("dipDays.csv")
dipDays = pd.DataFrame(dipDays["date"])
dipDays["dipDay"] = 1
dipDays


,date,dipDay
0,2012-04-16,1
1,2012-04-26,1
2,2012-04-29,1
3,2012-04-30,1
4,2012-05-01,1
5,2012-05-02,1
6,2012-05-07,1
7,2012-05-08,1
8,2012-05-09,1
9,2012-05-10,1


In [13]:
##############################
# This takes about 5 minutes #
##############################

mergedData2 = londonData.merge(dipDays, left_on='dayString', right_on='date', how='left')
mergedData2['dipDay'] = mergedData2['dipDay'].fillna(0).astype(int)


In [14]:
# Drop the extra columns
mergedData2.drop(columns=['date'], inplace=True)
mergedData2.head()

,LCLid,energy(kWh/hh),isNA,year,month,day,hour,minute,dayNumber,dayString,weekend,CustomerCount,dipDay
0,MAC000002,0,0,2012,10,12,0,30,4,2012-10-12,0,5261,0
1,MAC000002,0,0,2012,10,12,1,0,4,2012-10-12,0,5261,0
2,MAC000002,0,0,2012,10,12,1,30,4,2012-10-12,0,5261,0
3,MAC000002,0,0,2012,10,12,2,0,4,2012-10-12,0,5261,0
4,MAC000002,0,0,2012,10,12,2,30,4,2012-10-12,0,5261,0


In [15]:
mergedData2["dipDay"].value_counts()

0    163981828
1      3806641
Name: dipDay, dtype: int64

In [16]:
# Save
path = "Data/pickles/"
mergedData2.to_pickle(path + "londonDataWithDips.pkl")

# ==== SAVE AND LOAD ====
I decided to save the data as a pkl here so that I could just run the cells below this point.

In [17]:
# Load

import pandas as pd
path = "Data/pickles/"
londonData = pd.read_pickle(path + "londonDataWithDips.pkl")

MemoryError: 

In [ ]:
londonData.head()

,LCLid,energy(kWh/hh),isNA,year,month,day,hour,minute,dayNumber,dayString,weekend,CustomerCount,dipDay
0,MAC000246,0.852,0,2012,4,16,0,0,0,2012-04-16,0,1548,1
1,MAC000246,0.367,0,2012,4,16,0,30,0,2012-04-16,0,1548,1
2,MAC000246,0.079,0,2012,4,16,1,0,0,2012-04-16,0,1548,1
3,MAC000246,0.088,0,2012,4,16,1,30,0,2012-04-16,0,1548,1
4,MAC000246,0.076,0,2012,4,16,2,0,0,2012-04-16,0,1548,1


In [ ]:
londonData["energy(kWh/hh)"] = londonData["energy(kWh/hh)"].astype(float)
print(londonData["energy(kWh/hh)"].min())
print(londonData["energy(kWh/hh)"].max())

0.0
6.2839999


### Blackout signature - mark hours before 0 power as signature days?

### Weather Features - add later to see if improves

## Visualize the features

## Fix the data imbalance

## Seperate features (x) from labels (y)

## Feature Reduction

## Normalize / Scale the Data

## Investigate Variance or Feature Importance

# STEP 3: MODEL TRAINING & BUILDING


## Split into Train and Test data

## Use a Model

## Perform a Hyperparameter Search

## Save the best model

# STEP 4: EVALUATE THE MODEL

## Look at Metrics - Ex) Precision, Recall, F1 score

## Plot a Confusion Matrix

# STEP 5: DEPLOY MODEL